## Custom Layers 

In [1]:
import tensorflow as tf

In [2]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [9]:
# Introduction to Lambda Layers
# tf.keras.layers.Lambda(lambda x:tf.abs(x))
from keras import backend as K
def my_relu(x):
    return K.maximum(-0.1, x)

model2 = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape = (28,28)),
    tf.keras.layers.Dense(128),
    tf.keras.layers.Lambda(my_relu),
    tf.keras.layers.Dense(10, activation = 'softmax')
    
])
model2.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
model2.fit(x_train, y_train,epochs = 5)
model2.evaluate(x_test, y_test)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 36s 604us/sample - loss: 0.2546 - accuracy: 0.9274
Epoch 2/5
60000/60000 [==============================] - 31s 520us/sample - loss: 0.1112 - accuracy: 0.9676
Epoch 3/5
60000/60000 [==============================] - 31s 518us/sample - loss: 0.0756 - accuracy: 0.9776
Epoch 4/5
60000/60000 [==============================] - 28s 472us/sample - loss: 0.0554 - accuracy: 0.9832- ETA: 1s - loss: 0.0
Epoch 5/5
10000/10000 [==============================] - 5s 480us/sample - loss: 0.0773 - accuracy: 0.9780


[0.07733687666975893, 0.978]

In [5]:
from tensorflow.keras.layers import Layer
class SimpleDense(Layer):
    
    def __init__(self, units = 32):
        super(SimpleDense, self).__init__()
        self.units = units
        
    def build(self, input_shape):#Create the state of the layer(weights)
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(name = 'kernel',
                             initial_value = w_init(shape=(input_shape[-1], self.units),
                                                   dtype = 'float32'),
                             trainable = True)
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(name = 'bias', initial_value = b_init(shape = (self.units,),dtype = 'float32'), trainable = True)
    def call(self, inputs): # Defines the computation from inputs to outputs
        return tf.matmul(inputs, self.w) + self.b
    
my_dense = SimpleDense(units = 1)
x = tf.ones((1,1))
y = my_dense(x)
print(my_dense.variables)

[<tf.Variable 'simple_dense/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[-0.07750624]], dtype=float32)>, <tf.Variable 'simple_dense/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]


In [6]:
import numpy as np
xs = np.array([-1.0,0.0,1.0,2.0,3.0,4.0], dtype=float)
ys = np.array([-3.0,-1.0,1.0,3.0,5.0,7.0], dtype = float)
my_layer = SimpleDense(units = 1)
model = tf.keras.Sequential([my_layer])
model.compile(optimizer = 'sgd', loss = 'mean_squared_error')
model.fit(xs, ys, epochs = 500, verbose = 0)
print(model.predict([10]))
print(my_layer.variables)

[[18.981548]]
[<tf.Variable 'sequential/simple_dense_1/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[1.9973257]], dtype=float32)>, <tf.Variable 'sequential/simple_dense_1/bias:0' shape=(1,) dtype=float32, numpy=array([-0.9917087], dtype=float32)>]


In [ ]:
# Activating the custom layer
class Simple_Dense(Layer):
    def __init__(self, units =32, activation =None):
        super(Simple_Dense, self).__init__()
        self.units = units
        self.activation = tf.keras.activations.get(activation)
        
    def build(self, input_shape):
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(name = 'kernel',
                             initial_value = w_init(shape=(input_shape[-1], self.units),
                                                   dtype = 'float32'),
                             trainable = True)
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(name = 'bias', initial_value = b_init(shape = (self.units,),dtype = 'float32'), trainable = True)
    def call(self, inputs): # Defines the computation from inputs to outputs
        return self.activation(tf.matmul(inputs, self.w) + self.b)

In [8]:
model3 = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape = (28,28)),
    Simple_Dense(128, activation = 'relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation = 'softmax')
])

model3.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
model3.fit(x_train, y_train, epochs = 50)
model3.evaluate(x_test, y_test)

NameError: name 'Simple_Dense' is not defined